In [1]:
import pandas as pd
import numpy as np

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="HIDDEN",timeout=200)

import pymysql

con = pymysql.connect('localhost', 'root', 'HIDDEN', 'HIDDEN')

<strong>This is the Jupyter Notebook file for querying the customer's unique city and country names from MySQL, then passing those values to GeoPy for parsing the latitude and longitude values. After the values are stored, it is then inserted into the database called "city_points" a long with the city names. GeoPy needs a combination of city and country to obtain geographic data.</strong>

In [2]:
def select_main():
    '''
    Returns a Dataframe containing unique city and country from the customer's table.
    
    Parameters: 
        None passed to function.
    
    Returns:
        Pandas DataFrame containing:
        RangeIndex: 69 entries, 0 to 68
        Data columns (total 2 columns):
         #   Column   Non-Null Count  Dtype 
        ---  ------   --------------  ----- 
         0   city     69 non-null     object
         1   country  69 non-null     object
        dtypes: object(2)
    '''
    #connect to db, select values, pass to variable and close
    con.connect()
    select = con.cursor()
    statement = 'select distinct city,country from customers;'
    select.execute(statement)
    data = select.fetchall()
    con.commit()
    
    #pass to DataFrame method with columns, return the value
    cities = pd.DataFrame(data).rename({0:'city',1:'country'},axis=1)
    
    return cities

In [3]:
#load into memory
cities = select_main()

In [4]:
def get_points(cities):
    '''
    Returns a Dataframe containing containing city, country a long with latitude/longitude from GeoPy.
    
    Parameters: 
        DataFrame (total 2 columns)
    
    Returns:
        Pandas DataFrame containing:
        RangeIndex: 69 entries, 0 to 68
        Data columns (total 4 columns):
         #   Column     Non-Null Count  Dtype  
        ---  ------     --------------  -----  
         0   city       69 non-null     object 
         1   country    69 non-null     object 
         2   latitude   69 non-null     float64
         3   longitude  69 non-null     float64
        dtypes: float64(2), object(2)
    '''
    #two lists, pass country+city into one variable. pass to GeoPy method, append values to lists.
    lat = []
    lon = []

    for city, country in cities.values:
        string = city +', '+ country
        coords = geolocator.geocode(string, language='en')
        lat.append(coords.latitude)
        lon.append(coords.longitude)
    
    #assign to columns
    cities['latitude'] = lat
    cities['longitude'] = lon
    
    #return the Dataframe with new columns
    return cities

In [5]:
#new DataFrame with geographic attributes, might take a minute or two
with_points = get_points(cities)

In [9]:
#load into the customly created city_points table, linked to the customer table in the schema. commit the insert
def insert_points(with_points):
    '''
    Accepts the DataFrame with city, latitude and longitude to insert the values into the mysql table.
    
    Parameters: 
        DataFrame (total 4 columns)
    
    Returns:
        None
    '''
    #connect to db
    con.connect()
    insert_main = con.cursor()

    #cycle through values and insert them
    for row in with_points[['city','latitude','longitude']].values:
        insert_main.callproc('w3_insert_points',(row[0],row[1],row[2]))

    #commit the insertion
    con.commit()

In [10]:
#insert values in DataFrame to db
insert_points(with_points)